In [1]:
import pandas as pd
import numpy as np
from sklearn.externals import joblib

In [2]:
df = joblib.load('raw_obentou.pkl')
df.head()

,datetime,y,week,soldout,name,kcal,remarks,event,payday,weather,precipitation,temperature
0,2013-11-18,90,月,0,厚切りイカフライ,NaN,NaN,NaN,NaN,快晴,--,19.8
1,2013-11-19,101,火,1,手作りヒレカツ,NaN,NaN,NaN,NaN,快晴,--,17.0
2,2013-11-20,118,水,0,白身魚唐揚げ野菜あん,NaN,NaN,NaN,NaN,快晴,--,15.5
3,2013-11-21,120,木,1,若鶏ピリ辛焼,NaN,NaN,NaN,NaN,快晴,--,15.2
4,2013-11-22,130,金,1,ビッグメンチカツ,NaN,NaN,NaN,NaN,快晴,--,16.1


In [3]:
df.shape

(207, 12)

In [4]:
# 重複レコードを削除しておく
df = df.drop_duplicates()
df.shape

(207, 12)

### 欠損値を補完

In [5]:
df = df.fillna({
    'kcal':df['kcal'].mean(),
    'remarks':'不明',
    'event':'不明',
    'payday':0.0
})
df.head()

,datetime,y,week,soldout,name,kcal,remarks,event,payday,weather,precipitation,temperature
0,2013-11-18,90,月,0,厚切りイカフライ,404.409639,不明,不明,0.0,快晴,--,19.8
1,2013-11-19,101,火,1,手作りヒレカツ,404.409639,不明,不明,0.0,快晴,--,17.0
2,2013-11-20,118,水,0,白身魚唐揚げ野菜あん,404.409639,不明,不明,0.0,快晴,--,15.5
3,2013-11-21,120,木,1,若鶏ピリ辛焼,404.409639,不明,不明,0.0,快晴,--,15.2
4,2013-11-22,130,金,1,ビッグメンチカツ,404.409639,不明,不明,0.0,快晴,--,16.1


In [6]:
df.isnull().sum()

datetime         0
y                0
week             0
soldout          0
name             0
kcal             0
remarks          0
event            0
payday           0
weather          0
precipitation    0
temperature      0
dtype: int64

In [7]:
# 紐づけたいデータフレームを適当に作成
# 意図的に[メンチカツ]の前後にスペースを入れた文字列を複数混ぜる
df1 = pd.DataFrame({
    'name':['メンチカツ ', ' メンチカツ', ' メンチカツ ', 'メンチカツ', 'タンドリーチキン', '手作りロースカツ', 'マーボ豆腐', '肉じゃが'],
    'flag':[0, 0, 0, 1, 0, 1, 0, 1]
})

df1

,name,flag
0,メンチカツ,0
1,メンチカツ,0
2,メンチカツ,0
3,メンチカツ,1
4,タンドリーチキン,0
5,手作りロースカツ,1
6,マーボ豆腐,0
7,肉じゃが,1


In [8]:
len(df1['name'].value_counts())# このデータフレームのレコード数が分母の値になる

8

## 2つのデータフレームを[name]カラムで左結合

In [9]:
merge_df = pd.merge(df1, df, on='name', how='left')
merge_df# スペースが含まれた[メンチカツ]は元ファイルと紐づかず元ファイル側のカラムが欠損値になることがわかる

,name,flag,datetime,y,week,soldout,kcal,remarks,event,payday,weather,precipitation,temperature
0,メンチカツ,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,メンチカツ,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,メンチカツ,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,メンチカツ,1,2014-3-5,110.0,水,1.0,393.000000,不明,不明,0.0,雨,1.5,6.1
4,メンチカツ,1,2014-4-1,125.0,火,1.0,377.000000,不明,不明,0.0,薄曇,--,16.1
5,メンチカツ,1,2014-5-8,59.0,木,0.0,405.000000,不明,不明,0.0,薄曇,--,21.0
6,メンチカツ,1,2014-5-14,64.0,水,1.0,430.000000,不明,不明,0.0,薄曇,--,26.0
7,メンチカツ,1,2014-7-30,58.0,水,0.0,441.000000,不明,不明,0.0,晴れ,--,30.6
8,メンチカツ,1,2014-9-5,43.0,金,0.0,380.000000,不明,不明,0.0,晴れ,--,30.2
9,タンドリーチキン,0,2013-12-13,111.0,金,1.0,404.409639,不明,不明,0.0,快晴,--,13.9


In [10]:
print(len(df1['name'].value_counts()))
print(len(merge_df['name'].value_counts()))

8
8


In [11]:
merge_df.isnull().sum()# 落としたのは3レコード

name             0
flag             0
datetime         3
y                3
week             3
soldout          3
kcal             3
remarks          3
event            3
payday           3
weather          3
precipitation    3
temperature      3
dtype: int64

In [12]:
# 重複レコードを削除
u_merge_df = merge_df.drop_duplicates(subset=['name'])
len(u_merge_df)

8

## 何割のデータが紐づいた(or落ちた)のかを確認


In [13]:
# 左結合した後のユニークレコード数から欠損値の数を引いた値を左結合時に左側に指定したデータフレームのユニークレコード数で割ることで算出できる
(len(u_merge_df) - 3)  / len(df1['name'].value_counts())

0.625